<a href="https://colab.research.google.com/github/AmberGodil/ML/blob/main/Notebooks/Half-life.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install deepchem
%pip install rdkit-pypi

import pandas as pd
import numpy as np
import deepchem as dc

from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import warnings

warnings.filterwarnings('ignore')


Skipped loading some Pytorch utilities, missing a dependency. No module named 'torch'


This module requires PyTorch to be installed.


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [49]:
df = pd.read_csv("../Data/half-life-dataset.csv")
df.drop('Unnamed: 4', axis=1)

,Molecule,PUBCHEM_COMPOUND_CID,PUBCHEM_IUPAC_INCHI,PUBCHEM_IUPAC_INCHIKEY,Half-Life in MLM minutes,Binary Classification
0,[Cl-1].S1C2=C3SC=4C=5N3C36[H].O=C2N2C1=C1C=7[N...,6444403,InChI=1S/C21H22N3OS2.ClH/c1-4-23-13-9-8-10-15(...,VSKYOTRJSLYFHX-UXJRWBAGSA-M,4.4,0
1,O1C23C45C67C89C%10%11C%12%13C%14%15C%16%17C8%1...,11686124,InChI=1S/C35H54N2O10/c1-19(6-9-29(36)40)24-7-8...,BDEAKWDBQDWAAQ-ORYUWPRUSA-N,38.0,0
2,O1C23C45C67C89C%10%11C%12%13C%14%15C%16%17C8%1...,11571413,InChI=1S/C43H72N4O10/c1-27(10-13-37(50)44-20-2...,SBDGPZKXHQPLJY-UEPJFKMZSA-N,120.0,1
3,O1C23C45C67C89C%10%11C%12%13C%14%15C%16%17C8%1...,11535162,InChI=1S/C47H62N2O10/c1-29(16-19-41(52)48-34-1...,ASQDPWRWIJWAFR-CBKGSJONSA-N,120.0,1
4,[Cl]C1=NC=2N3N=C4N=2.[F]C25[F].[F]2.[F]C2=C6C1...,16103018,"InChI=1S/C18H18ClF5N6O.ClH/c1-9(18(22,23)24)28...",HAFUDZOLFLXVIY-FVGYRXGTSA-N,433.0,1
...,...,...,...,...,...,...
889,N12C34C56N7C89C1%10[H].N1%11C%12%13C%14%15C%16...,54585781,InChI=1S/C13H24N6/c1-10(2)9-15-11-8-12(17-13(1...,LJUOJSBZZTZROX-UHFFFAOYSA-N,4.0,0
890,[Cl]C=1C2=C3C=4N=C5N=C6N7C89C%10%11N%12C%13=NC...,44595856,InChI=1S/C16H17ClN8/c17-11-3-1-2-10(8-11)12-9-...,UIACBYZKCZCWAB-UHFFFAOYSA-N,9.0,0
891,N12C34C56N7C89C1%10[H].N1=C2C2=C%11N=C1N1[H].C...,25129523,InChI=1S/C15H19N5/c1-19-7-9-20(10-8-19)14-11-1...,NIJGWJIOMPHDBP-UHFFFAOYSA-N,4.0,0
892,[Cl]C1=C2C=3C4=C5N6C=3C3=C1[H].O=C5N1C57C89N%1...,4908365,InChI=1S/C14H16ClN3O/c1-17-4-6-18(7-5-17)14(19...,HUQJRYMLJBBEDO-UHFFFAOYSA-N,2.0,0


Modify Inchi for row 561 and 679. Original Inchi causing issue when converting to Mol.

In [50]:
new_inchi_561 = 'InChI=1S/C22H12F4N4O/c23-18-4-3-15(8-17(18)22(24,25)26)30-20(31)6-2-13-9-27-19-5-1-12(7-16(19)21(13)30)14-10-28-29-11-14/h1-11H,(H,28,29)'
new_inchi_679 = 'InChI=1S/C25H30N4O2/c30-25(27-31)14-11-21-9-12-23-22(19-21)26-24(13-10-20-7-3-1-4-8-20)29(23)18-17-28-15-5-2-6-16-28/h1,3-4,7-9,11-12,14,19,31H,2,5-6,10,13,15-18H2,(H,27,30)/b14-11+'

df.at[561, 'PUBCHEM_IUPAC_INCHI'] = new_inchi_561
df.at[679, 'PUBCHEM_IUPAC_INCHI'] = new_inchi_679

df.to_csv('modified_half-life.csv', index=False)

print("Modified value:", df.at[561, 'PUBCHEM_IUPAC_INCHI'])
print("Modified value:", df.at[679, 'PUBCHEM_IUPAC_INCHI'])

print("CSV file copied and modified successfully.")

Modified value: InChI=1S/C22H12F4N4O/c23-18-4-3-15(8-17(18)22(24,25)26)30-20(31)6-2-13-9-27-19-5-1-12(7-16(19)21(13)30)14-10-28-29-11-14/h1-11H,(H,28,29)
Modified value: InChI=1S/C25H30N4O2/c30-25(27-31)14-11-21-9-12-23-22(19-21)26-24(13-10-20-7-3-1-4-8-20)29(23)18-17-28-15-5-2-6-16-28/h1,3-4,7-9,11-12,14,19,31H,2,5-6,10,13,15-18H2,(H,27,30)/b14-11+
CSV file copied and modified successfully.


Convert each molecule to mol from Inchi and then from mol to SMILES

In [95]:
df['mol'] = df['PUBCHEM_IUPAC_INCHI'].apply(Chem.MolFromInchi)
df.drop('Unnamed: 4', axis=1)
df['SMILES'] = df['mol'].apply(lambda x: Chem.MolToSmiles(x) if x is not None else None)

Split dataset into 80% trainning 20% testing

In [70]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Binary Classification'])

# Save the training set to a CSV file
train_df.to_csv("../data/half-life-train.csv", index=False)

# Save the testing set to a CSV file
test_df.to_csv("../data/half-life-test.csv", index=False)

print("Training and testing sets created and saved as separate CSV files.")

Training and testing sets created and saved as separate CSV files.


In [72]:
DATA_DIR = "../Data"

TRN_PATH = f"{DATA_DIR}/half-life-train.csv"
TST_PATH = f"{DATA_DIR}/half-life-test.csv"


In [87]:
df_trn = pd.read_csv(TRN_PATH)
df_tst = pd.read_csv(TST_PATH)

In [88]:
print(df_tst['Binary Classification'].value_counts(normalize=True))
print(df_trn['Binary Classification'].value_counts(normalize=True))

0    0.709497
1    0.290503
Name: Binary Classification, dtype: float64
0    0.706294
1    0.293706
Name: Binary Classification, dtype: float64


Simplify dataframes so they only include SMILES and Binary Classification columns

In [99]:
df_trn= df_trn[['SMILES', 'Binary Classification']]
df_tst= df_tst[['SMILES', 'Binary Classification']]


Using deepchem to generate features: RDKit descriptors and circular fingerprints

In [100]:
featurizers = {
    'circular': dc.feat.CircularFingerprint(size=2048, radius=4), 
    'rdkit': dc.feat.RDKitDescriptors()
}

def featurize(df, key, featurizer):
    feats = featurizer.featurize(df['SMILES'])
    pd_feats = pd.DataFrame(feats, columns=[key + '_' + str(i+1) for i in range(feats.shape[1])])
    return pd.concat([df, pd_feats], axis=1)

for k, f in featurizers.items():
    print(f"Generating {k} fingerprints...")
    df_trn = featurize(df_trn, k, f)
    df_tst = featurize(df_tst, k, f)
print("Done.")

print(f"Shape of trn set = {df_trn.shape}")
print(f"Shape of tst set = {df_tst.shape}")

Generating circular fingerprints...
Generating rdkit fingerprints...
Done.
Shape of trn set = (715, 2258)
Shape of tst set = (179, 2258)


In [101]:
df_trn.columns

Index(['SMILES', 'Binary Classification', 'circular_1', 'circular_2',
       'circular_3', 'circular_4', 'circular_5', 'circular_6', 'circular_7',
       'circular_8',
       ...
       'rdkit_199', 'rdkit_200', 'rdkit_201', 'rdkit_202', 'rdkit_203',
       'rdkit_204', 'rdkit_205', 'rdkit_206', 'rdkit_207', 'rdkit_208'],
      dtype='object', length=2258)

In [102]:
X_trn, y_trn = df_trn.drop(['SMILES', 'Binary Classification'], axis=1), df_trn['Binary Classification']
X_tst, y_tst = df_tst.drop(['SMILES', 'Binary Classification'], axis=1), df_tst['Binary Classification']

Scale dataset StandardScaler

In [103]:
scaler = StandardScaler()
X_trn = scaler.fit_transform(X_trn)
X_tst = scaler.transform(X_tst)


Generate random forest model

In [104]:
rf = RandomForestClassifier(n_estimators= 1000, max_depth=14, max_samples=80, random_state=42)

Fit data to model

In [105]:
rf.fit(X_trn, y_trn)

RandomForestClassifier(max_depth=14, max_samples=80, n_estimators=1000,
                       random_state=42)

In [106]:
rf.score(X_tst, y_tst)

0.7597765363128491

In [107]:
rf_preds = rf.predict(X_tst)


In [108]:
print(classification_report(y_tst, rf_preds))

              precision    recall  f1-score   support

           0       0.76      0.97      0.85       127
           1       0.76      0.25      0.38        52

    accuracy                           0.76       179
   macro avg       0.76      0.61      0.61       179
weighted avg       0.76      0.76      0.71       179



Second model using Hyperparameter tuning

In [109]:
rf2 = RandomForestClassifier()

In [110]:
param_grid = [{
    'n_estimators': [100, 500, 1000],
    'max_depth': [5, 10, 15, 20],
    'max_samples': [50, 80, 100]
    #'random_state': [42]
}]

Using GridSearchCV to select optimal parameters for model

In [111]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(rf2, param_grid, cv=2, scoring='accuracy', n_jobs=-1)

In [112]:
grid_search.fit(X_trn, y_trn)

GridSearchCV(cv=2, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid=[{'max_depth': [5, 10, 15, 20],
                          'max_samples': [50, 80, 100],
                          'n_estimators': [100, 500, 1000]}],
             scoring='accuracy')

In [113]:
print("Grid search best score", grid_search.best_score_)
print("Grid search best params",grid_search.best_params_)

Grid search best score 0.7328803029591725
Grid search best params {'max_depth': 20, 'max_samples': 100, 'n_estimators': 100}


Binary Classification Metrics


In [116]:
print("Accuracy score: ", accuracy_score(y_tst, grid_search.predict(X_tst)))
print("ROC AUC score: ", roc_auc_score(y_tst, grid_search.predict(X_tst)))
print("Average precision score: ", average_precision_score(y_tst, grid_search.predict(X_tst)))

Accuracy score:  0.7430167597765364
ROC AUC score:  0.5917625681405209
Average precision score:  0.37730984099699183
